In [1]:
import os, sys
sys.path.insert(0, os.path.abspath("../../../SNN_WMA/src")) 
print(sys.path)

['C:\\Users\\luiggi\\Documents\\GitSites\\SNN_WMA\\src', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313', '', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\win32', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\luiggi\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\Pythonwin']


In [2]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np

# Biblioteca y módulos de flopy
import flopy
from flopy.plot.styles import styles

# Extras para mf6 y flopy (módulo creado en este proyecto)
import xmf6

from modflowapi import ModflowApi

In [3]:
mesh = xmf6.MeshDis(
    nrow = 1,    # Number of rows
    ncol = 120,  # Number of columns
    nlay = 1,    # Number of layers
    row_length = 12.0,    # Length of system ($cm$)
    column_length = 0.1,  # Length of system ($cm$)
    top = 1.0,   # Top of the model ($cm$)
    bottom = 0,  # Layer bottom elevation ($cm$)
)

xmf6.nice_print(mesh.get_dict(), 'Space discretization')

tdis = xmf6.TDis(
    perioddata = ((120, 1.0, 1.0),) # PERLEN, NSTP, TSMULT
)
xmf6.nice_print(tdis, 'Time discretization')

## Model Units
ml_units = {"time" : "seconds",
            "length" : "centimeters"}
xmf6.nice_print(ml_units, 'Units')

## Physical parameters
ph_par = {}
ph_par["specific_discharge"] = 0.1  # Specific discharge ($cm s^{-1}$)
ph_par["hydraulic_conductivity"] = 0.01  # Hydraulic conductivity ($cm s^{-1}$)
ph_par["source_concentration"] = 1.0  # Source concentration (unitless)
xmf6.nice_print(ph_par, 'Physical parameters')

os_par = dict(
    ws = os.getcwd() + '/output', # Ruta del espacio de trabajo
    mf6_exe = os.path.abspath('../bin/mf6.exe'), # Ejecutable
    flow_name = 'flow', # Nombre de la simulación
)
xmf6.nice_print(os_par, 'MODFLOW 6 environment')

oc_par = dict(
    head_file = f"{os_par['flow_name']}.hds",
    fbudget_file = f"{os_par['flow_name']}.bud"
)
xmf6.nice_print(oc_par, 'Output files')


Space discretization
------------------------------
          row_length = 12.0      
          col_length = 0.1       
          lay_length = 1.0       
                ncol = 120       
                nrow = 1         
                nlay = 1         
                delr = 0.1       
                delc = 0.1       
                dell = 1.0       
                 top = 1.0       
              bottom = 0.0       

Time discretization
------------------------------
      NPER = 1       
TOTAL TIME = 120.00

PER     PERLEN     NSTP     TSMULT        Dt1
  1     120.00       1.00       1.00   1.20e+02

Units
------------------------------
                time = seconds   
              length = centimeters

Physical parameters
------------------------------
  specific_discharge = 0.1       
hydraulic_conductivity = 0.01      
source_concentration = 1.0       

MODFLOW 6 environment
------------------------------
                  ws = C:\Users\luiggi\Documents\GitSites\MODFLOW_A

In [4]:
sim = flopy.mf6.MFSimulation(
    sim_name = os_par['flow_name'], 
    sim_ws   = os_par['ws'], 
    exe_name = os_par['mf6_exe']
)
print(sim)

flopy.mf6.ModflowTdis(
    sim, 
    nper = tdis.nper(), 
    perioddata = tdis.perioddata(), 
    time_units = ml_units["time"]
)

ims = flopy.mf6.ModflowIms(sim)
print(ims)

gwf = flopy.mf6.ModflowGwf(
    sim, 
    modelname  = os_par['flow_name'], 
    save_flows = True
)
print(gwf)

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units = ml_units["length"],
    nlay = mesh.nlay,
    nrow = mesh.nrow,
    ncol = mesh.ncol,
    delr = mesh.delr,
    delc = mesh.delc,
    top  = mesh.top,
    botm = mesh.bottom,
)
print(dis)

ic = flopy.mf6.ModflowGwfic(
    gwf, 
    strt=1.0 # Initial head
) 
print(ic)


npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge = True,
    save_saturation = True,
    icelltype = 0,
    k = ph_par['hydraulic_conductivity'],
)
print(npf)

chd = flopy.mf6.ModflowGwfchd(
    gwf, 
    stress_period_data=[[(0, 0, mesh.ncol - 1), 1.0]]  # Node, Constant value
) 
print(chd)


q   = ph_par["specific_discharge"] * mesh.delc * mesh.delr * mesh.top
aux = ph_par["source_concentration"]

wel = flopy.mf6.ModflowGwfwel(
    gwf,
    stress_period_data = [[(0, 0, 0), q, aux,]],
    pname = "WEL-1",
    auxiliary = ["CONCENTRATION"],
)

print(wel)

oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord   = oc_par['head_file'],
    budget_filerecord = oc_par['fbudget_file'],
    saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")],
)
print(oc)

sim.write_simulation()

sim_name = flow
sim_path = C:\Users\luiggi\Documents\GitSites\MODFLOW_API\emsdatasets\Test\output
exe_name = C:\Users\luiggi\Documents\GitSites\MODFLOW_API\emsdatasets\bin\mf6.exe

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = flow



package_name = ims_-1
filename = flow.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = flow


name = flow
model_type = gwf6
version = mf6
model_relative_path = .


package_name = dis
filename = flow.dis
package_type = dis
model_or_simulation_package = model
model_name = flow

Block options
--------------------
length_units
{internal}
(centimeters)


Block dimensions
--------------------
nlay
{internal}
(1)

nrow
{internal}
(1)

ncol
{internal}
(120)


Block griddata
--------------------
delr
{constant 0.1}

delc
{constant 0.1}

top
{constant 1.0}

botm
{constant 0.0}



package_name = ic


In [5]:
if sys.platform == "win32":
    mf6_dll = "libmf6.dll"
elif sys.platform == "darwin":
    mf6_dll = "libmf6.dylib"
else:
    mf6_dll = "libmf6.so"

mf6_dll = os.path.abspath(os.path.join("..", "bin", mf6_dll))
print(sys.platform, ": ", mf6_dll)

win32 :  C:\Users\luiggi\Documents\GitSites\MODFLOW_API\emsdatasets\bin\libmf6.dll


In [12]:
mf6_config_file = os.path.join(os_par['ws'], 'mfsim.nam')
mf6 = ModflowApi(mf6_dll, working_directory=os_par['ws'])

# initialize the model
mf6.initialize(mf6_config_file)

#print("MF6 Input var names :", mf6.get_input_var_names())

#for label in mf6.get_input_var_names():
#    if 'FLOW' in label:
#        print(label)
        
# time loop
current_time = mf6.get_current_time()
end_time = mf6.get_end_time()

print(current_time)
print(end_time)


dis_address = mf6.get_var_address("NDIM", os_par['flow_name'].upper(), "DIS")
dismine = mf6.get_value(dis_address)
print(dismine)

print(mf6.get_value(mf6.get_var_address("YC", os_par['flow_name'].upper(), "DIS")))

print(mf6.get_value(mf6.get_var_address("X", os_par['flow_name'].upper())))


0.0
120.0
[3]
[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
sim.run_simulation()

FloPy is using the following executable to run the model: ..\..\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.6.0 12/20/2024

   MODFLOW 6 compiled Dec 31 2024 17:10:16 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Government shall

(False, [])

In [8]:
# Obtenemos los resultados de la carga hidráulica
head = flopy.utils.HeadFile(
    os.path.join(os_par['ws'], 
                 oc_par['head_file'])).get_data()

# Obtenemos los resultados del BUDGET
bud  = flopy.utils.CellBudgetFile(
    os.path.join(os_par['ws'], 
                 oc_par['fbudget_file']),
    precision='double'
)
# Obtenemos las velocidades
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

# Verificamos el tipo y dimensiones de los arreglos donde
# están almacenados la carga hidráulica, el BUDGET, y la velocidad.
print('Head : ', type(head), head.shape)
print('Budget : ', type(bud), bud.shape)
print('spdis : ', type(spdis), spdis.shape)
print('qx : ', type(qx), qx.shape)
print('qy : ', type(qy), qy.shape)
print('qz : ', type(qz), qz.shape)

print(head.shape, '\n', head)

ValueError: datafile error: file is empty: C:\Users\luiggi\Documents\GitSites\MODFLOW_API\emsdatasets\Test/output\flow.hds

In [9]:
with styles.USGSPlot():
    plt.rcParams['font.family'] = 'DeJavu Sans'
    x, _, _ = mesh.get_coords()
    plt.figure(figsize=(10,3))
    plt.plot(x, head[0, 0], marker=".", ls ="-", mec="blue", mfc="none", markersize="1", label = 'Head')
    plt.xlim(0, 12)
    plt.xticks(ticks=np.linspace(0, mesh.row_length,13))
    plt.xlabel("Distance (cm)")
    plt.ylabel("Head")
    plt.legend()
    plt.grid()
    plt.show()

NameError: name 'head' is not defined

<Figure size 1000x300 with 0 Axes>

In [ ]:
plt.figure(figsize=(10,0.15))
ax = plt.gca()
pmv0 = flopy.plot.PlotMapView(gwf, ax=ax)
pmv0.plot_grid(colors='dimgray', lw=0.5)
plt.yticks(ticks=[0, 0.1],fontsize=8)

plt.show()